**Task 04: Graph querying**

In [ ]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/formacion_REALE/main"

In [ ]:
from rdflib import Graph, Namespace, Literal
g = Graph()
g.parse(github_storage+"/rdf/example3.rdf", format="xml")

List all resources with property VCARD:FN

In [ ]:
from rdflib.plugins.sparql import prepareQuery


VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")


q1 = prepareQuery('''
  SELECT ?Subject WHERE { 
    ?Subject vcard:FN ?FullName. 
  }
  ''',
  initNs = { "vcard": VCARD}
)


for r in g.query(q1):
  print(r.Subject)

Let's repeat the previous query but asking for the full name of the resources

In [ ]:
q2 = prepareQuery('''
  SELECT ?Subject ?FullName WHERE { 
    ?Subject vcard:FN ?FullName.
  } 
  ''',
  initNs = { "vcard": VCARD}
)

for r in g.query(q2):
  print(r.Subject, r.FullName)

Retrieve all resources that have "Smith" as family name

In [ ]:
from rdflib import XSD

q3 = prepareQuery('''
  SELECT ?Subject WHERE { 
    ?Subject vcard:Family ?Family.
  } 
  ''',
  initNs = { "vcard": VCARD}
)

for r in g.query(q3, initBindings = {'?Family' : Literal('Smith', datatype=XSD.string)}):
  print(r.Subject)

Obtain all elements with an email

In [ ]:
from rdflib import FOAF

q4 = prepareQuery('''
  SELECT ?Subject ?Email WHERE { 
    ?Subject foaf:email ?Email.
  } 
  ''',
  initNs = { "foaf": FOAF}
)

for r in g.query(q4):
  print(r.Subject,r.Email)

Who knows (FOAF:knows)  "Jane Smith"? Retrieve their name (VCARD:Given)

In [ ]:
q5 = prepareQuery('''
  SELECT  ?Subject ?Given  WHERE {
    ?Subject foaf:knows ?JaneSmith.
	?JaneSmith vcard:FN ?JaneSmithFullName.
	?Subject vcard:Given ?Given.
  }  
  ''',
  initNs = { "foaf": FOAF, "vcard": VCARD, "xsd":XSD}
)

for r in g.query(q5, initBindings = {'?JaneSmithFullName' : Literal('Jane Smith', datatype=XSD.string)}):
  print(r.Subject, r.Given)

Now let's try to query an external SPARQL endpoint. Query DBpedia to retrieve the top 10 countries with more politicians.

We need another library called sparql wrapper

In [ ]:
!pip install SPARQLWrapper

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)
sparql.setQuery("""
    select ?bp (count(?p) as ?num_politicians) 
    WHERE {
        ?p a <http://dbpedia.org/ontology/Politician>.
        ?p <http://dbpedia.org/ontology/birthPlace> ?bp.
        ?bp a <http://dbpedia.org/ontology/Country> 
    }group by ?bp order by desc (?num_politicians) limit 10
""")

try:
    for result in sparql.queryAndConvert()["results"]["bindings"]:
      print(result['bp']['value'],result['num_politicians']['value'])
except Exception as e:
    print(e)